**写在前面**：本节内容是 [Datawhale三月的组队学习 - 集成学习（上）- CH2-机器学习基础模型回顾 -【Task6 掌握分类问题的评估及超参数调优】](https://github.com/datawhalechina/team-learning-data-mining/blob/master/EnsembleLearning/CH2-%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0%E5%9F%BA%E7%A1%80%E6%A8%A1%E5%9E%8B%E5%9B%9E%E9%A1%BE/%E7%AC%AC%E4%BA%8C%E7%AB%A0%EF%BC%9A%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0%E5%9F%BA%E7%A1%80.ipynb) 的学习笔记，对应notebook的2.2（4）节，学习周期2天

## 导入库和数据

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection  import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import StandardScaler # 标准化
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import uniform  # 引入均匀分布设置参数
from sklearn.decomposition import PCA
from sklearn.svm import SVC

### 练习

In [5]:
# 读入数据
lfw_dataset = datasets.fetch_lfw_people(min_faces_per_person=70)
lfw_data = lfw_dataset['data']
lfw_image = lfw_dataset['images']
lfw_target = lfw_dataset['target']
lfw_targetName = lfw_dataset['target_names']

In [6]:
lfw_data.shape
# 维度太大了，先做降维再建模

(1288, 2914)

In [19]:
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(lfw_data, lfw_target, test_size=0.25, random_state=328)

In [23]:
# 降维
pca = PCA(n_components=150, random_state=328).fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [37]:
# 训练并网格搜索调参
pipe_svc = Pipeline([("StandardScaler",StandardScaler()), ("svc",SVC(random_state=328))])
param_range = [0.0001,0.001,0.01,0.1,1.0,10.0,100.0,1000.0]
param_grid = [{'svc__C':param_range,'svc__kernel':['linear']},
              {'svc__C':param_range,'svc__gamma':param_range,'svc__kernel':['rbf']}]
gs = GridSearchCV(pipe_svc, 
                  param_grid,
                  scoring='accuracy', 
                  cv=10)
gs = gs.fit(X_train_pca, y_train)
print(gs.best_score_)
print(gs.best_params_)
# 预测与评分
y_pred = gs.predict(X_test_pca)
print('MSE:', MSE(y_test, y_pred))

0.8187929553264605
{'svc__C': 10.0, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}
MSE: 1.0372670807453417


In [39]:
# 训练并随机搜索调参
pipe_svc = Pipeline([("StandardScaler",StandardScaler()), ("svc",SVC(random_state=328))])
param_range = uniform(loc=0, scale=100)
param_grid = [{'svc__C':param_range,'svc__kernel':['linear']},
              {'svc__C':param_range,'svc__gamma':param_range,'svc__kernel':['rbf']}]
rs = RandomizedSearchCV(pipe_svc, 
                        param_grid,
                        cv=10)
rs = rs.fit(X_train_pca, y_train)
print(rs.best_score_)
print(rs.best_params_)
# 预测与评分
y_pred = rs.predict(X_test_pca)
print('MSE:', MSE(y_test, y_pred))

0.7732817869415807
{'svc__C': 71.62932046579327, 'svc__kernel': 'linear'}
MSE: 1.186335403726708
